In [1]:
from numpy import genfromtxt
import subprocess
import os
import numpy as np
import sys
prefix = '/Users/yuding/Dropbox/GitHub/Multi_Region_CliPP/intermediate/'
current_dir = '/Users/yuding/Dropbox/GitHub/Multi_Region_CliPP/'
path_for_preprocess= prefix
snv_input = 'R1_snv.txt'
cn_input = 'R1_cna.txt'
purity_input = 'R1_purity.txt'
run_preprocess = os.path.join(current_dir, "preprocess.R")


In [2]:
p_preprocess = subprocess.Popen(["Rscript", run_preprocess, snv_input, cn_input, purity_input, 'xxx', path_for_preprocess], stdout=subprocess.PIPE, stderr=subprocess.PIPE)


In [3]:
import pandas as pd
r = genfromtxt(prefix+"r.txt", delimiter="\t")
n = genfromtxt(prefix+"n.txt", delimiter="\t")
minor = genfromtxt(prefix+"minor.txt", delimiter="\t")
total = genfromtxt(prefix+"total.txt", delimiter="\t")
purity = genfromtxt(prefix+"purity_ploidy.txt", delimiter="\t")
major = total - minor 
major[major == 0] = minor[major == 0]
df = pd.DataFrame({
    'mutation' : [i for i in range(len(r))],
    'region' : ['R1' for i in range(len(r))],
    'ref_counts' : n - r,
    'alt_counts' : r,
    'normal_cn' : [2 for i in range(len(r))],
    'major_cn' : major,
    'minor_cn' : minor,
    'tumour_purity' : [purity for i in range(len(r))]
})

In [1]:
from snv import *

In [2]:
import pandas as pd

# File path
file_path = '/Users/yuding/Dropbox/GitHub/Multi_Region_CliPP/multi_clipp_simulation_data/simulation_data_cluster_5_region_1_read_depth_100_replica_2.tsv'

# Read the data into a DataFrame
df = pd.read_csv(file_path, sep='\t')

In [3]:
rho = 0.8
omega = 1
n = df.shape[0]
m = 1
gamma = 0.01
control_large = 5
precision = 0.01
max_iteration = 20
res = CliPP2(df, rho, 0.01, omega, n, m)

 12 , 0.008943276705263514

In [4]:
sets = {i for i in range(n)}
combinations_2 = list(itertools.combinations(sets, 2))
pairs_mapping = {combination: index for index, combination in enumerate(combinations_2)}
pairs_mapping_inverse = {index: combination for index, combination in enumerate(combinations_2)}

# Get matrices
read_mat = get_read_mat(df)
total_read_mat = get_total_read_mat(df)
c_mat = get_c_mat(df)
bb_mat = get_b_mat(df)
tumor_cn_mat = get_tumor_cn_mat(df)
linearApprox = get_linear_approximation(c_mat)
# Initialize variables
#12/12/2024
phi_hat = (read_mat / (total_read_mat * c_mat))
scale_parameter = np.max([1, np.max(phi_hat)])
phi_hat = phi_hat / scale_parameter
phi_hat[phi_hat > sigmoid(control_large)] = sigmoid(control_large)
phi_hat[phi_hat < sigmoid(-control_large)] = sigmoid(-control_large)
p = inverse_sigmoid(phi_hat)
p[p > control_large] = control_large
p[p < -control_large] = -control_large
p = p.reshape([n * m])
v = np.zeros([len(combinations_2) * m])
for i in range(len(combinations_2)):
    pair = combinations_2[i]
    index_v = pairs_mapping[pair]
    start_v = index_v * m
    end_v = (index_v + 1) * m
    l1, l2 = pairs_mapping_inverse[index_v]
    a_mat = a_mat_generator(l1, l2, n, m)
    v[start_v: end_v] = matmul_by_torch(a_mat, p)
    
y = np.ones([len(combinations_2) * m])
omega = np.ones([len(combinations_2)])
k = 0

control_large = 5
wcut = np.array(linearApprox[0])
coef = np.array(linearApprox[1])
temp = 100
# ADMM
while k < max_iteration and precision < temp:
    p = update_p(p, v, y, n, m, read_mat, total_read_mat, bb_mat, tumor_cn_mat, coef, wcut, combinations_2, pairs_mapping, rho, control_large)
    temp = 0
    for i in range(len(combinations_2)):
        pair = combinations_2[i]
        index_v = pairs_mapping[pair]
        start_v = index_v * m
        end_v = (index_v + 1) * m
        v[start_v: end_v] = update_v_SCAD(index_v, pairs_mapping_inverse, p, y, n, m, rho, omega[i], gamma)                
        y[start_v: end_v] = update_y(y[start_v: end_v], v[start_v: end_v], i, pairs_mapping_inverse, p, n, m, rho)
        l1, l2 = pairs_mapping_inverse[index_v]
        a_mat = a_mat_generator(l1, l2, n, m)
        temp = max(temp, np.linalg.norm(matmul_by_torch(a_mat, p) - v[start_v: end_v]))
    rho = 1.02 * rho
    k = k + 1
    print('\r', k, ',', temp, end="")
        
diff = np.zeros((n, n))
class_label = -np.ones(n)
class_label[0] = 0
group_size = [1]
labl = 1
least_mut = 25
for i in range(1, n):
    for j in range(i):
        index_v = pairs_mapping[(j, i)]
        start_v = index_v * m
        end_v = (index_v + 1) * m
        diff[j, i] = np.linalg.norm(v[start_v: end_v]) if np.linalg.norm(v[start_v: end_v]) > 0.05 else 0
        diff[i, j] = diff[j, i]
for i in range(1, n):
    for j in range(i):
        if diff[j, i] == 0:
            class_label[i] = class_label[j]
            group_size[int(class_label[j])] += 1
            break
    if class_label[i] == -1:
        class_label[i] = labl
        labl += 1
        group_size.append(1)
        
# quality control
tmp_size = np.min(np.array(group_size)[np.array(group_size) > 0])
tmp_grp = np.where(group_size == tmp_size)
refine = False
if tmp_size < least_mut:
    refine = True
while refine:
    refine = False
    tmp_col = np.where(class_label == tmp_grp[0][0])[0]
    for i in range(len(tmp_col)):
        if tmp_col[i] != 0 and tmp_col[i] != n - 1:
            tmp_diff = np.abs(np.append(np.append(diff[0:tmp_col[i], tmp_col[i]].T.ravel(), 100),
                                        diff[tmp_col[i], (tmp_col[i] + 1):n].ravel()))
            tmp_diff[tmp_col] += 100
            diff[0:tmp_col[i], tmp_col[i]] = tmp_diff[0:tmp_col[i]]
            diff[tmp_col[i], (tmp_col[i] + 1):n] = tmp_diff[(tmp_col[i] + 1):n]
        elif tmp_col[i] == 0:
            tmp_diff = np.append(100, diff[0, 1:n])
            tmp_diff[tmp_col] += 100
            diff[0, 1:n] = tmp_diff[1:n]
        else:
            tmp_diff = np.append(diff[0:(n - 1), n - 1], 100)
            tmp_diff[tmp_col] += 100
            diff[0:(n - 1), n - 1] = tmp_diff[0:(n - 1)]
        ind = tmp_diff.argmin()
        group_size[class_label.astype(np.int64, copy=False)[tmp_col[i]]] -= 1
        class_label[tmp_col[i]] = class_label[ind]
        group_size[class_label.astype(np.int64, copy=False)[tmp_col[i]]] += 1
    tmp_size = np.min(np.array(group_size)[np.array(group_size) > 0])
    tmp_grp = np.where(group_size == tmp_size)
    refine = False
    if tmp_size < least_mut:
        refine = True
labels = np.unique(class_label)

phi_out = np.zeros((len(labels), m))
for i in range(len(labels)):
    ind = np.where(class_label == labels[i])[0]
    class_label[ind] = i
    phi_out[i, :] = np.sum(phi_hat[ind,: ] * total_read_mat[ind,: ]) / np.sum(total_read_mat[ind, :])
    

if len(labels) > 1:
    phi_norm = np.linalg.norm(phi_out, axis=1)
    sort_phi = np.sort(phi_norm)
    indices = [np.where(phi_norm == element)[0][0] for element in sort_phi]
    phi_diff = sort_phi[1:] - sort_phi[:-1]
    min_val = phi_diff.min()
    min_ind = phi_diff.argmin()
    while min_val < 0.01:
        combine_ind = np.where(phi_out == sort_phi[indices[min_ind]])[0]
        combine_to_ind = np.where(phi_out == sort_phi[indices[min_ind] + 1])[0]
        class_label[class_label == combine_ind] = combine_to_ind
        labels = np.unique(class_label)
        phi_out = np.zeros(len(labels))
        for i in range(len(labels)):
            ind = np.where(class_label == labels[i])[0]
            class_label[ind] = i
            phi_out[i, :] = np.sum(phi_hat[ind, :] * total_read_mat[ind, :]) / np.sum(total_read_mat[ind, :])
        if len(labels) == 1:
            break
        else:
            phi_norm = np.linalg.norm(phi_out, axis=1)
            sort_phi = np.sort(phi_norm)
            indices = [np.where(phi_norm == element)[0][0] for element in sort_phi]
            phi_diff = sort_phi[1:] - sort_phi[:-1]
            min_val = phi_diff.min()
            min_ind = phi_diff.argmin()
phi_res = np.zeros((n, m))
for lab in range(len(phi_out)):
    phi_res[class_label == lab, :] = phi_out[lab, :]

purity = get_purity_mat(df)[0,0]

 12 , 0.008943276705263514

In [5]:
p

array([ 0.02866593, -4.61069999,  0.52860803, -2.83398465,  0.59516325,
        0.78490021, -1.79113702, -1.49320806, -0.09314153, -4.53657572,
       -4.55947801, -0.06843625, -0.76784407, -4.58974126, -1.49902166,
       -4.53246242, -0.25607762, -0.78936333, -0.54968096, -4.43678508,
       -1.86668438, -0.76326936, -4.47817989, -4.49056287, -0.5387675 ,
       -1.15797412, -1.75555749,  0.36989234,  0.04502931, -2.39792415,
       -4.54908117, -4.55998207,  0.08603517, -4.545519  , -0.46187327,
       -0.04000605, -0.42217977, -1.18749636, -4.43239923, -4.47883452,
       -1.34768017, -4.5611802 , -0.5939559 , -0.39604233,  0.03250871,
       -2.02552441, -4.44490029, -1.35829348, -0.00923671, -4.54426428,
       -0.0063589 ,  0.46295584, -0.45558295,  0.19295973, -0.60211808,
        0.40265498, -1.96040693,  0.88569082,  0.24867918,  0.49874695,
       -0.08980151, -0.94457626, -1.17058598, -1.18328234, -0.420129  ,
       -0.19375052,  0.11716477,  0.25936357, -1.80100645, -1.67

In [5]:
phi_res

array([[0.54696766],
       [0.08264033],
       [0.54696766],
       [0.08264033],
       [0.54696766],
       [0.54696766],
       [0.08264033],
       [0.08264033],
       [0.54696766],
       [0.08264033],
       [0.08264033],
       [0.54696766],
       [0.34219993],
       [0.08264033],
       [0.08264033],
       [0.08264033],
       [0.34219993],
       [0.34219993],
       [0.34219993],
       [0.08264033],
       [0.08264033],
       [0.34219993],
       [0.08264033],
       [0.08264033],
       [0.34219993],
       [0.34219993],
       [0.08264033],
       [0.54696766],
       [0.54696766],
       [0.08264033],
       [0.08264033],
       [0.08264033],
       [0.54696766],
       [0.08264033],
       [0.34219993],
       [0.54696766],
       [0.34219993],
       [0.34219993],
       [0.08264033],
       [0.08264033],
       [0.08264033],
       [0.08264033],
       [0.34219993],
       [0.34219993],
       [0.54696766],
       [0.08264033],
       [0.08264033],
       [0.082

In [6]:
sets = {i for i in range(n)}
combinations_2 = list(itertools.combinations(sets, 2))
pairs_mapping = {combination: index for index, combination in enumerate(combinations_2)}
pairs_mapping_inverse = {index: combination for index, combination in enumerate(combinations_2)}

# Get matrices
read_mat = get_read_mat(df)
total_read_mat = get_total_read_mat(df)
c_mat = get_c_mat(df)
bb_mat = get_b_mat(df)
tumor_cn_mat = get_tumor_cn_mat(df)
linearApprox = get_linear_approximation(c_mat)
# Initialize variables
#12/12/2024
phi_hat = (read_mat / (total_read_mat * c_mat))
scale_parameter = np.max([1, np.max(phi_hat)])
phi_hat = phi_hat / scale_parameter
phi_hat[phi_hat > sigmoid(control_large)] = sigmoid(control_large)
phi_hat[phi_hat < sigmoid(-control_large)] = sigmoid(-control_large)
p = inverse_sigmoid(phi_hat)
p[p > control_large] = control_large
p[p < -control_large] = -control_large
p = p.reshape([n * m])
v = np.zeros([len(combinations_2) * m])

for i in range(len(combinations_2)):
    pair = combinations_2[i]
    index_v = pairs_mapping[pair]
    start_v = index_v * m
    end_v = (index_v + 1) * m
    l1, l2 = pairs_mapping_inverse[index_v]
    a_mat = a_mat_generator(l1, l2, n, m)
    v[start_v: end_v] = matmul_by_torch(a_mat, p)
    
y = np.ones([len(combinations_2) * m])
omega = np.ones([len(combinations_2)])
k = 0

control_large = 5
wcut = np.array(linearApprox[0])
coef = np.array(linearApprox[1])
temp = 100

In [7]:
No_mutation = n * m
theta_hat = np.reshape(read_mat / total_read_mat, [No_mutation])

theta = np.exp(p) * np.reshape(bb_mat, [No_mutation]) / (2 + np.exp(p) * np.reshape(tumor_cn_mat, [No_mutation]))

A = np.sqrt(np.reshape(total_read_mat, [No_mutation])) * (
            (p <= wcut[:, 0]) * coef[:, 1] + (p >= wcut[:, 1]) * coef[:, 5] + (p > wcut[:, 0]) * (
                p < wcut[:, 1]) * coef[:, 3] - theta_hat) / np.sqrt(theta * (1 - theta))
B = np.sqrt(np.reshape(total_read_mat, [No_mutation])) * (
            (p <= wcut[:, 0]) * coef[:, 0] + (p >= wcut[:, 1]) * coef[:, 4] + (p > wcut[:, 0]) * (
                p < wcut[:, 1]) * coef[:, 2]) / np.sqrt(theta * (1 - theta))

linear = rho * get_v_mat(v, y, rho, combinations_2, pairs_mapping, n, m) - (B * A)

Minv = 1 / (B ** 2 + No_mutation * rho)
Minv_diag = np.diag(Minv)

trace_g = -rho * np.sum(Minv)

Minv_outer = np.outer(Minv,Minv)
inverted = Minv_diag - (1 / (1 + trace_g) * (-rho) * Minv_outer)
p_new    = matmul_by_torch(inverted, linear.T)
p_new = p_new.reshape([No_mutation])
p_new[p_new > control_large] = control_large
p_new[p_new < -control_large] = -control_large
temp = 0
for i in range(len(combinations_2)):
    pair = combinations_2[i]
    index_v = pairs_mapping[pair]
    start_v = index_v * m
    end_v = (index_v + 1) * m
    v[start_v: end_v] = update_v_SCAD(index_v, pairs_mapping_inverse, p, y, n, m, rho, omega[i], gamma)                
    y[start_v: end_v] = update_y(y[start_v: end_v], v[start_v: end_v], i, pairs_mapping_inverse, p, n, m, rho)
    l1, l2 = pairs_mapping_inverse[index_v]
    a_mat = a_mat_generator(l1, l2, n, m)
    temp = max(temp, np.linalg.norm(matmul_by_torch(a_mat, p) - v[start_v: end_v]))
rho = 1.02 * rho
k = k + 1
print('\r', k, ',', temp, end="")


 1 , 0.9956164694770704

In [8]:
p

array([ 0.05059903, -5.        ,  0.47548959, -2.77258872,  0.53386717,
        0.71479891, -1.80747645, -1.43540407, -0.09097178, -5.        ,
       -5.        , -0.08072177, -0.6991532 , -5.        , -1.44862827,
       -5.        , -0.22314355, -0.69314718, -0.48550782, -5.        ,
       -1.90423745, -0.69314718, -5.        , -5.        , -0.44631151,
       -1.04596856, -1.7176515 ,  0.30623657,  0.02105341, -2.39789527,
       -5.        , -5.        ,  0.06963852, -5.        , -0.42121347,
       -0.0334915 , -0.38946477, -1.11718046, -5.        , -5.        ,
       -1.3077885 , -5.        , -0.59401444, -0.40804309, -0.01333353,
       -2.05838813, -5.        , -1.33758809, -0.05975804, -5.        ,
       -0.02000067,  0.37806613, -0.470423  ,  0.12451707, -0.59532999,
        0.31719613, -2.00372972,  0.77711256,  0.19766065,  0.40546511,
       -0.11778304, -0.88503819, -1.08570888, -1.11365017, -0.42380136,
       -0.23602636,  0.06309785,  0.18232156, -1.84299683, -1.67